# Fighter Clustering
Looking to categorize fighters into fighting styles (grappler, striker, balanced) which I will use in my eda and classifier/predictor

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('processed_fighter_data.csv')
df = df.dropna()

In [3]:
df.head()

,fighter_name,wins,losses,draws,consec_wins,height_in,weight_lbs,reach_in,stance,DOB,...,SLpM,str_acc,SApM,str_def,td_avg,td_acc,td_def,sub_avg,avg_ctrl,weight_class
0,Shamil Abdurakhimov,20,5,0,0,75,235,76,Orthodox,1981-09-02,...,2.45,0.44,2.45,0.58,1.23,0.24,0.47,0.2,92.25,Heavyweight
1,Daichi Abe,6,2,0,0,61,170,71,Orthodox,1991-11-27,...,3.80,0.33,4.49,0.56,0.33,0.50,0.00,0.0,7.33,Welterweight
2,Klidson Abreu,15,4,0,0,72,205,74,Orthodox,1992-12-24,...,2.05,0.40,2.90,0.55,0.64,0.20,0.80,0.0,62.00,Light Heavyweight
3,Juan Adams,5,3,0,0,77,265,80,Orthodox,1992-01-16,...,7.09,0.55,4.06,0.34,0.91,0.66,0.57,0.0,90.00,Heavyweight
4,Anthony Adams,8,2,0,0,73,185,76,Orthodox,1988-01-13,...,3.17,0.41,5.93,0.44,0.00,0.00,0.00,0.0,0.00,Middleweight


## Method 1
Manually creating a training sample and using knn to cluster the fighters

In [4]:
df['style'] = None

In [5]:
strikers = ['Anderson Silva','Connor McGregor', 'Mirko Filipovic',
           'Jose Aldo', 'Lyoto Machida', 'Alistair Overeem', 'Israel Adesanya',
           'Stephen Thompson', 'Edson Barboza', 'Max Holloway',
           'Anthony Pettis','Robbie Lawler', 'Donald Cerrone',
           'Francis Ngannou','Carlos Condit', 'Stipe Miocic',
           'Cris Cyborg', 'Alexander Gustafsson', 'Jorge Masvidal', 'Leon Edwards']

balanced = ['Valentina Shevchenko', 'Georges St-Pierre', 'TJ Dillashaw', 'Gilbert Burns', 'Tony Ferguson', 'Chan Sung Jung', 'Demetrious Johnson',
           'Jon Jones', 'Nate Diaz']

grapplers = ['Demian Maia', 'Khabib Nurmagomedov', 'Ronaldo Souza','Daniel Cormier', 'Charles Oliveira', 'Ben Askren',
            'Roger Gracie', 'Brian Ortega', 'Henry Cejudo', 'Yoel Romero',
            'Colby Covington','Randy Couture','Brock Lesnar','Aljamain Sterling', 'Curtis Blaydes']

In [6]:
for fighter in strikers:
    df['style'].loc[df['fighter_name']==fighter] = 0
    
for fighter in balanced:
    df['style'].loc[df['fighter_name']==fighter] = 1

for fighter in grapplers:
    df['style'].loc[df['fighter_name']==fighter] = 2

C:\Users\danie\python_stuff\Anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
#getting all the indices so we can train the model
fighters = [fighter for fighter in strikers]
for fighter in balanced:
    fighters.append(fighter)
for fighter in grapplers:
    fighters.append(fighter)

indices = []
for fighter in fighters:
    try:
        indices.append(df.loc[df['fighter_name']==fighter].index[0])
    except:
        None
df_subset = df.loc[indices, :]
df_subset['style'] = df_subset['style'].astype('int64')

In [8]:
df_imputed = df
#using knn classifier to predict fighting style on fight statistics`
KNN = KNeighborsClassifier(n_neighbors=5)
KNN.fit(df_subset.loc[:,'SLpM':'avg_ctrl'], df_subset['style'])
predictions = KNN.predict(df_imputed.loc[:,'SLpM':'sub_avg'])
df_imputed['style'] = predictions

df_imputed['style'] = df_imputed['style'].map({0:'striker', 1:'balanced', 2:'grappler'}).astype('category')
#leaving off here, imputation by knn seems ok
df_imputed['style'].value_counts()

ValueError: query data dimension must match training data dimension

In [ ]:
grappler_check = []
balanced_check = []
striker_check = []
for row in range(len(df_imputed)):
    if df_imputed['style'][row] == 'grappler':
        grappler_check.append(df_imputed['fighter_name'][row])
    elif df_imputed['style'][row] == 'balanced':
        balanced_check.append(df_imputed['fighter_name'][row])
    else:
        striker_check.append(df_imputed.fighter_name[row])

In [ ]:
#grappler_check
#balanced_check

In [ ]:
df_imputed.loc[df_imputed['fighter_name']=='Zabit Magomedsharipov']

In [ ]:
df_imputed['style'].value_counts()

In [ ]:
df['avg_ctrl'].isna().value_counts()